In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read data
df = pd.read_csv('data-sets/air_passengers.csv')

# Create lagged features
for i in range(1, 3):
    df[f'lag_{i}'] = df['Passengers'].shift(i)

# Drop NA values
df.dropna(inplace=True)

# Define inputs and target
X = df[['lag_1', 'lag_2']].values
y = df['Passengers'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)  # 0.2 x 0.8 = 0.16


In [2]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
import torch
from torch import nn

class LSTM(pl.LightningModule):
    def __init__(self, input_size, hidden_layer_size, num_layers, lr, weight_decay, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.lr = lr
        self.weight_decay = weight_decay
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.loss_func = nn.MSELoss()

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), -1))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_func(y_hat, y)
        return loss


In [3]:

# Parameters for grid search
learning_rates = [0.01, 0.1]
epochs = [50, 100]
hidden_sizes = [50, 100]
layers = [1, 3]
weight_decays = [1e-3, 1e-2]

from torch.utils.data import DataLoader, TensorDataset
# Preparing data loaders
train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
val_dataset = TensorDataset(torch.Tensor(X_val), torch.Tensor(y_val))
test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Initialize minimum loss and best model
min_loss = float('inf')
best_model = None
best_params = None

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

print(X_val)
# Iterate over hyperparameters
for lr in learning_rates:
    for epoch in epochs:
        for hidden_size in hidden_sizes:
            for layer in layers:
                for weight_decay in weight_decays:
                    model = LSTM(input_size=2, hidden_layer_size=hidden_size, num_layers=layer, lr=lr, weight_decay=weight_decay)
                    trainer = Trainer(max_epochs=epoch)
                    trainer.fit(model, train_loader, val_loader)
                    model.eval()
                    with torch.no_grad():
                        predictions = model(X_val_tensor)
                        loss = model.loss_func(predictions, y_val_tensor)
                        if loss.item() < min_loss:
                            min_loss = loss.item()
                            best_model = model
                            best_params = {'lr': lr, 'epoch': epoch, 'hidden_size': hidden_size, 'layer': layer, 'weight_decay': weight_decay}

print("Best model parameters:", best_params)
print("Best model minimum loss:", min_loss)

[[230. 218.]
 [149. 125.]
 [622. 535.]
 [461. 419.]
 [313. 317.]
 [363. 348.]
 [272. 264.]
 [148. 148.]
 [171. 166.]
 [315. 306.]
 [132. 118.]
 [146. 162.]
 [196. 196.]
 [306. 355.]
 [150. 145.]
 [420. 396.]
 [362. 407.]
 [183. 181.]
 [201. 180.]
 [413. 374.]
 [269. 267.]
 [204. 201.]
 [188. 204.]]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\cemer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")

  | Name      | Type    | Params
--------------------------------------
0 | lstm      | LSTM    | 10.8 K
1 | linear    | Linear  | 51    
2 | loss_func | MSELoss | 0     
--------------------------------------
10.9 K    Trainable params
0         Non-trainable params
10.9 K    Total params
0.043     Total estimated model params size (MB)
C:\Users\cemer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Py

Training: 0it [00:00, ?it/s]

C:\Users\cemer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\cemer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([26])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=50` reached.
C:\Users\cemer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.

Training: 0it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type    | Params
--------------------------------------
0 | lstm      | LSTM    | 203 K 
1 | linear    | Linear  | 101   
2 | loss_func | MSELoss | 0     
--------------------------------------
203 K     Trainable params
0         Non-trainable params
203 K     Total params
0.813     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


Best model parameters: {'lr': 0.1, 'epoch': 100, 'hidden_size': 50, 'layer': 1, 'weight_decay': 0.01}
Best model minimum loss: 15290.4716796875
